In [1]:
import pyodbc
from import_function import import_scoring_data, import_data
from features_by_customer_type import customer_type_features
conn = pyodbc.connect(dsn='VerticaProd')
x=120
ct='New'
response=['ABANDONED']

cont_features, bool_features, catag_features, cont_score_features, bool_score_features, catag_score_features=customer_type_features(ct)
scoring_data = "SELECT A.* FROM (SELECT SUP_REMIC_TAXABLE_INCOME,	SUP_REMIC_SCHQ_INCOME,	SUP_REMIC_EXCESS_INCLUSION,	SUP_RE_TOTAL_INCOME,	SUP_RE_PROFESSIONAL_INCOME,	SUP_RE_PERSONAL_DAYS,	SUP_RE_INCOME_ROYALTIES,	SUP_RE_INCOME_RENTS,	SUP_RE_EXPENSES_UTILITIES,	SUP_RE_EXPENSES_TRAVEL,	SUP_RE_EXPENSES_TOTAL,	SUP_RE_EXPENSES_TAXES,	SUP_RE_EXPENSES_SUPPLIES,	SUP_RE_EXPENSES_REPAIRS,	SUP_RE_EXPENSES_OTHER_INTEREST,	SUP_RE_EXPENSES_MORTGAGE_INTEREST,	SUP_RE_EXPENSES_MANAGEMENT,	SUP_RE_EXPENSES_LEGAL,	SUP_RE_EXPENSES_INSURANCE,	SUP_RE_EXPENSES_DEPRECIATION,	SUP_RE_EXPENSES_DEDUCTIBLE_LOSS,	SUP_RE_EXPENSES_COMMISSIONS,	SUP_RE_EXPENSES_CLEANING,	SUP_RE_EXPENSES_ADVERTISING,	SUP_PS_TOTAL_INCOME,	SUP_PS_SEC179_EXPENSE_DEDUCTION,	SUP_PS_PY_LOSS,	SUP_PS_PASSIVE_LOSS,	SUP_PS_PASSIVE_INCOME,	SUP_PS_NONPASSIVE_LOSS,	SUP_PS_NONPASSIVE_INCOME,	SUP_PS_AT_RISK,	SUP_FARM_RENTAL_INCOME,	SUP_FARM_GROSS_INCOME,	SUP_EST_TOTAL_INCOME,	SUP_EST_PASSIVE_LOSS,	SUP_EST_PASSIVE_INCOME,	SUP_EST_NONPASSIVE_LOSS,	SUP_EST_NONPASSIVE_INCOME,	NUM_EXEMPTIONS,	NUM_DEPENDENTS,	FLAG_OLD_OR_BLIND,	BUS_WILL_FILE_1099,	BUS_VEHICLE_MILES_OTHER,	BUS_VEHICLE_MILES_COMMUTE,	BUS_VEHICLE_MILES_BUSINESS,	BUS_STATUTORY_EMPLOYEE,	BUS_START_ACQUIRE,	BUS_REQUIRE_1099,	BUS_OTHER_INCOME,	BUS_NET_PROFIT,	BUS_MATERIAL_PARTICIPATE,	BUS_INVESTMENT_AT_RISK,	BUS_GROSS_PROFIT,	BUS_GROSS_INCOME,	BUS_EXPENSE_WAGES,	BUS_EXPENSE_UTILITIES,	BUS_EXPENSE_USE_OF_HOME,	BUS_EXPENSE_TRAVEL,	BUS_EXPENSE_TOTAL,	BUS_EXPENSE_TAXES,	BUS_EXPENSE_SUPPLIES,	BUS_EXPENSE_REPAIRS,	BUS_EXPENSE_RENT_VEHICLES,	BUS_EXPENSE_RENT_OTHER,	BUS_EXPENSE_PROFIT_TENTATIVE,	BUS_EXPENSE_PENSION,	BUS_EXPENSE_OTHER_INTEREST,	BUS_EXPENSE_OTHER,	BUS_EXPENSE_OFFICE,	BUS_EXPENSE_MORTGAGE_INTEREST,	BUS_EXPENSE_MEALS,	BUS_EXPENSE_LEGAL,	BUS_EXPENSE_INSURANCE,	BUS_EXPENSE_EMPLOYEE_BENEFITS,	BUS_EXPENSE_DEPRECIATION,	BUS_EXPENSE_DEPLETION,	BUS_EXPENSE_CONTRACT_LABOR,	BUS_EXPENSE_COMMISSIONS,	BUS_EXPENSE_CAR,	BUS_EXPENSE_ADVERTISING,	BUS_COGS_TOTAL,	BUS_COGS_SUPPLIES,	BUS_COGS_PURCHASES,	BUS_COGS_LABOR,	BUS_COGS_INVENTORY_START,	BUS_COGS_INVENTORY_END,	BUS_COGS,	AMOUNT_UNREPORTED_SS_MEDICARE_TAX,	AMOUNT_UNEMPLOYMENT,	AMOUNT_TUITION,	AMOUNT_TOTAL_TAX,	AMOUNT_TOTAL_PAYMENTS,	AMOUNT_TOTAL_INTEREST_PAID,	AMOUNT_TOTAL_INCOME,	AMOUNT_TOTAL_DEDUCTIONS,	AMOUNT_TOTAL_DEDUCTIBLE_EXPENSES,	AMOUNT_TOTAL_CREDITS,	AMOUNT_TAXES_PAID,	AMOUNT_TAXABLE_SOCIAL_SEC,	AMOUNT_TAXABLE_OFFSETS,	AMOUNT_TAXABLE_IRA,	AMOUNT_TAXABLE_INTEREST,	AMOUNT_TAXABLE_INCOME,	AMOUNT_TAX_PREP_FEES,	AMOUNT_TAX_DUE,	AMOUNT_TAX_CREDITS,	AMOUNT_TAX,	AMOUNT_STUDENT_LOAN_INTEREST_DEDUCTION,	AMOUNT_STATE_LOCAL_TAX,	AMOUNT_STATE_LOCAL_SALES_TAX,	AMOUNT_SOCIAL_SEC,	AMOUNT_SELF_EMPLOYMENT_TAX,	AMOUNT_SELF_EMPLOYMENT_RETIREMENT,	AMOUNT_SELF_EMPLOYMENT_HEALTH_INSURANCE,	AMOUNT_SCHE,	AMOUNT_SALARIES_AND_WAGES,	AMOUNT_RETIREMENT_SAVINGS_CREDIT,	AMOUNT_RESIDENTIAL_ENERGY_CREDIT,	AMOUNT_REFUND,	AMOUNT_REAL_ESTATE_TAX,	AMOUNT_QUALIFIED_DIVIDENDS,	AMOUNT_PERSONAL_PROPERTY_TAXES,	AMOUNT_PAID_WITH_EXTENSION,	AMOUNT_OTHER_TAXES,	AMOUNT_OTHER_PAYMENTS,	AMOUNT_OTHER_INCOME,	AMOUNT_OTHER_GAIN,	AMOUNT_OTHER_DEDUCTIBLE_TAXES,	AMOUNT_OTHER_DEDUCTIBLE_EXPENSES,	AMOUNT_OTHER_CREDITS,	AMOUNT_ORDINARY_DIVIDENDS,	AMOUNT_NT_COMBAT_PAY,	AMOUNT_MOVING_EXPENSE,	AMOUNT_MORTGAGE_POINTS_NON_1098,	AMOUNT_MORTGAGE_INTEREST_NON_1098,	AMOUNT_MORTGAGE_INTEREST,	AMOUNT_MORTGAGE_INSURANCE,	AMOUNT_MISC_DEDUCTIONS,	AMOUNT_MEDICAL_DENTAL_EXPENSES_DEDUCTION,	AMOUNT_MEDICAL_DENTAL_EXPENSES,	AMOUNT_IRA_DISTRIBUTIONS,	AMOUNT_IRA_DEDUCTION,	AMOUNT_INVESTMENT_INTEREST_PAID,	AMOUNT_INCOME_TAX_WITHHELD,	AMOUNT_INCOME_TAX,	AMOUNT_HSA,	AMOUNT_HOPE_CREDIT,	AMOUNT_HOMEBUYER_CREDIT_REPAYMENT,	AMOUNT_FUEL_TAX_CREDIT,	AMOUNT_FOREIGN_TAX_CREDIT,	AMOUNT_FARM_INCOME,	AMOUNT_EXPENSES_DEDUCTION,	AMOUNT_EXEMPTIONS,	AMOUNT_EXCESS_SS_RRTA_WITHHELD,	AMOUNT_ESTIMATED_TAX_PENALTY,	AMOUNT_ESTIMATED_TAX,	AMOUNT_EMPLOYEE_EXPENSES,	AMOUNT_EITC,	AMOUNT_EDUCATION_CREDIT,	AMOUNT_EARLY_WITHDRAWAL_PENALTY,	AMOUNT_DOMESTIC_PRODUCTION_DEDUCTION,	AMOUNT_DISABLED_CREDIT,	AMOUNT_DEDUCTIBLE_SELF_EMPLOYMENT_TAX,	AMOUNT_CHILD_CREDIT,	AMOUNT_CHILD_CARE_CREDIT,	AMOUNT_CHARITABLE_CONTRIBUTIONS_NONCASH,	AMOUNT_CHARITABLE_CONTRIBUTIONS_CASH,	AMOUNT_CHARITABLE_CONTRIBUTIONS_CARRYOVER,	AMOUNT_CHARITABLE_CONTRIBUTIONS,	AMOUNT_CERTAIN_BUSINESS_EXPENSE,	AMOUNT_CASUALTY_LOSSES,	AMOUNT_CAPITAL_GAIN,	AMOUNT_BUSINESS_INCOME,	AMOUNT_AMT,	AMOUNT_ALIMONY_PAID,	AMOUNT_ALIMONY_INCOME,	AMOUNT_ADJUSTMENTS,	AGI,	AGE_TAXPAYER,	AGE_SPOUSE,	AGE_DEPENDENT_MIN,	AGE_DEPENDENT_MAX,	AGE_DEPENDENT_AVG,	CUSTOMER_KEY,	ABANDONED,	TTO_FLAG,	STATE_ATTACH_COUNT,	MINDBENDER_FLAG,	CA_REFUND_TRANSFER_FLAG,	NON_CA_REFUND_TRANSFER_FLAG,	REFUND_TRANSFER_FLAG,	CA_AUDIT_DEFENSE_FLAG,	NON_CA_AUDIT_DEFENSE_FLAG,	AUDIT_DEFENSE_FLAG,	CA_MAX_FLAG,	NON_CA_MAX_FLAG,	MAX_FLAG,	PS_FLAG,	PRS_SCORE,	REJECT_COUNT,	TAX_WEEK,	TAX_DAY,	CORE_FLAG,	REQUIRED_TAKE_FLAG,	FSCHC_FLAG,	FSCHCEZ_FLAG,	FSCHE_FLAG,	FSCHA_FLAG,	FSCHD_FLAG,	FSCHF_FLAG,	MISC1099_FLAG,	ACCEPTED_EFILE,	NUM_CARE_CONTACTS,	RT_FLAG,	PRE_UPVOTES,	PRE_DOWNVOTES,	PRE_TOTAL_VOTES,	POST_UPVOTES,	POST_DOWNVOTES,	POST_TOTAL_VOTES,	SUP_RE_WILL_FILE_1099,	SUP_RE_TYPE_SINGLE_FAMILY,	SUP_RE_TYPE_SHORT_RENTAL,	SUP_RE_TYPE_SELF_RENTAL,	SUP_RE_TYPE_ROYALTIES,	SUP_RE_TYPE_OTHER,	SUP_RE_TYPE_MULTI_FAMILY,	SUP_RE_TYPE_LAND,	SUP_RE_TYPE_COMMERCIAL,	SUP_RE_REQUIRE_1099,	SUP_PS_SCORP,	SUP_PS_PARTNERSHIP_FOREIGN,	SUP_PS_PARTNERSHIP,	NUM_W2,	NUM_SCHE,	NUM_SCHC,	FLAG_ITEMIZED_DEDUCTIONS,	BUS_VEHICLE_PERSONAL_OFFDUTY,	BUS_VEHICLE_PERSONAL_ANOTHER,	BUS_COGS_METHOD_OTHER,	BUS_COGS_METHOD_COST_OR_MARKET,	BUS_COGS_METHOD_COST,	BUS_ACCOUNTING_METHOD_OTHER,	BUS_ACCOUNTING_METHOD_CASH,	BUS_ACCOUNTING_METHOD_ACCRUAL,	student_taxpayer,	_taxpayer,	retired_taxpayer,	unemployed_taxpayer,	cashier_taxpayer,	teacher_taxpayer,	manager_taxpayer,	sales_taxpayer,	customerservice_taxpayer,	laborer_taxpayer,	cook_taxpayer,	server_taxpayer,	retail_taxpayer,	engineer_taxpayer,	military_taxpayer,	cna_taxpayer,	salesassociate_taxpayer,	registerednurse_taxpayer,	truckdriver_taxpayer,	driver_taxpayer,	mechanic_taxpayer,	nurse_taxpayer,	waitress_taxpayer,	supervisor_taxpayer,	labor_taxpayer,	construction_taxpayer,	warehouse_taxpayer,	administrativeassistant_taxpayer,	accountant_taxpayer,	customerservicerep_taxpayer,	clerk_taxpayer,	receptionist_taxpayer,	medicalassistant_taxpayer,	disabled_taxpayer,	softwareengineer_taxpayer,	assistantmanager_taxpayer,	officemanager_taxpayer,	electrician_taxpayer,	technician_taxpayer,	none_taxpayer,	machineoperator_taxpayer,	caregiver_taxpayer,	employed_taxpayer,	bartender_taxpayer,	projectmanager_taxpayer,	maintenance_taxpayer,	welder_taxpayer,	housekeeper_taxpayer,	homemaker_taxpayer,	foodservice_taxpayer,	selfemployed_taxpayer,	chef_taxpayer,	csr_taxpayer,	warehouseworker_taxpayer,	factoryworker_taxpayer,	attorney_taxpayer,	consultant_taxpayer,	policeofficer_taxpayer,	carpenter_taxpayer,	secretary_taxpayer,	rn_taxpayer,	banker_taxpayer,	operator_taxpayer,	worker_taxpayer,	security_taxpayer,	production_taxpayer,	socialworker_taxpayer,	securityofficer_taxpayer,	pharmacytechnician_taxpayer,	machinist_taxpayer,	barista_taxpayer,	analyst_taxpayer,	deliverydriver_taxpayer,	stocker_taxpayer,	factory_taxpayer,	dentalassistant_taxpayer,	custodian_taxpayer,	generalmanager_taxpayer,	employee_taxpayer,	salesmanager_taxpayer,	retailmanager_taxpayer,	housekeeping_taxpayer,	storemanager_taxpayer,	securityguard_taxpayer,	na_taxpayer,	painter_taxpayer,	accountmanager_taxpayer,	correctionalofficer_taxpayer,	lpn_taxpayer,	operationsmanager_taxpayer,	bankteller_taxpayer,	fastfood_taxpayer,	manufacturing_taxpayer,	crewmember_taxpayer,	generallabor_taxpayer,	plumber_taxpayer,	marketing_taxpayer,	dispatcher_taxpayer,	hostess_taxpayer,	management_taxpayer,	janitor_taxpayer,	homemaker_spouse,	retired_spouse,	unemployed_spouse,	teacher_spouse,	student_spouse,	housewife_spouse,	none_spouse,	disabled_spouse,	home_maker_spouse,	house_wife_spouse,	stayathomemom_spouse,	nurse_spouse,	manager_spouse,	registerednurse_spouse,	sales_spouse,	customerservice_spouse,	cashier_spouse,	na_spouse,	laborer_spouse,	retail_spouse,	officemanager_spouse,	administrativeassistant_spouse,	engineer_spouse,	secretary_spouse,	accountant_spouse,	cook_spouse,	receptionist_spouse,	truckdriver_spouse,	self_employed_spouse,	mechanic_spouse,	rn_spouse,	server_spouse,	cna_spouse,	driver_spouse,	supervisor_spouse,	medicalassistant_spouse,	clerk_spouse,	socialworker_spouse,	projectmanager_spouse,	salesassociate_spouse,	mother_spouse,	waitress_spouse,	military_spouse,	construction_spouse,	attorney_spouse,	electrician_spouse,	dentalassistant_spouse,	stayathomemother_spouse,	customerservicerep_spouse,	mom_spouse,	substituteteacher_spouse,	consultant_spouse,	maintenance_spouse,	policeofficer_spouse,	banker_spouse,	labor_spouse,	welder_spouse,	softwareengineer_spouse,	na_spouse2,	technician_spouse,	marketing_spouse,	assistantmanager_spouse,	bankteller_spouse,	stayathomeparent_spouse,	caregiver_spouse,	warehouse_spouse,	educator_spouse,	carpenter_spouse,	bookkeeper_spouse,	housekeeper_spouse,	physician_spouse,	clerical_spouse,	analyst_spouse,	paralegal_spouse,	pharmacist_spouse,	machineoperator_spouse,	chef_spouse,	physicaltherapist_spouse,	humanresources_spouse,	lpn_spouse,	foodservice_spouse,	pharmacytechnician_spouse,	executiveassistant_spouse,	officeassistant_spouse,	disable_spouse,	salesmanager_spouse,	accountmanager_spouse,	professor_spouse,	hairstylist_spouse,	machinist_spouse,	selfemployed_spouse,	custodian_spouse,	factoryworker_spouse,	retailmanager_spouse,	director_spouse,	graphicdesigner_spouse,	bartender_spouse,	administrator_spouse,	home_spouse,	accounting_spouse,	unemployeed_spouse,	ORDER_AMOUNT,	FEDERAL_REVENUE,	MINDBENDER_REVENUE,	CA_REFUND_TRANSFER_REVENUE,	NON_CA_REFUND_TRANSFER_REVENUE,	REFUND_TRANSFER_REVENUE,	CA_AUDIT_DEFENSE_REVENUE,	NON_CA_AUDIT_DEFENSE_REVENUE,	AUDIT_DEFENSE_REVENUE,	CA_MAX_REVENUE,	NON_CA_MAX_REVENUE,	MAX_REVENUE,	PS_REVENUE,	FLAG_ITEMIZE_SEPARATELY,	TOTAL_REVENUE,	STATE_REVENUE,	START_SKU,	ENTRY_PAGE_GROUP,	CHANNEL,	IMPORT_TYPE,	START_SKU_ROLLUP,	FED_FORM_TYPE,	PRODUCT_ROLLUP,	PRODUCT_EDITION_DESCRIPTION,	CUSTOMER_TYPE_ROLLUP,	CUSTOMER_TYPE,	CUSTOMER_DEFINITION,	CUSTOMER_DEFINITION_ADJ,	NEW_CUSTOMER_DEFINITION,	COMPLETED_SKU,	LAST_STATUS,	FILING_STATUS,	DMA_AREA FROM  CTG_ANALYTICS_WS.SM_RETENTION_MODEL  WHERE   TAX_YEAR=2016 AND CUSTOMER_DEFINITION_ADJ='NEW TO TURBOTAX' )A inner join (SELECT DISTINCT CUSTOMER_KEY FROM  CTG_ANALYTICS_WS.SM_RETENTION_MODEL  WHERE   TAX_YEAR=2016 AND TAX_DAY=120) B ON A.CUSTOMER_KEY=B.CUSTOMER_KEY limit 100"
data = "SELECT * FROM (SELECT * FROM  CTG_ANALYTICS_WS.SM_RETENTION_MODEL  WHERE   TAX_YEAR=2015 AND  CUSTOMER_DEFINITION_ADJ IN ('NEW TO TURBOTAX')    )A   order by random() limit 500000"
df_no_pca = import_data(data, cont_features, bool_features,response, catag_features)
scoring_df=import_scoring_data(scoring_data, cont_score_features, bool_score_features,catag_score_features)





c:\python27\lib\site-packages\pandas\core\frame.py:2762: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


df_cont done
data_scaled done
just_dummies done
df_trans done
new feature list done


import_function.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_bool.drop([f], axis=1, inplace=True)


In [2]:
from bin_cont_features import bin_pca
j='True'
response='ABANDONED'
df_pca=df_no_pca
dummy_pca, length_dict = bin_pca(df_pca,response, j)
print list(dummy_pca), dummy_pca.head()
print dict

['<=0.5', '>0.5', 'between 0.5 and 166.0', '>=166.0']
['<=-23765.0', '>-23765.0', 'between -23765.0 and 3819.5', '>=3819.5']
['<=0.5', '>0.5', 'between 0.5 and 4.5', '>=4.5']
['<=0.5', '>0.5', 'between 0.5 and 227.5', '>=227.5']
['<=40.0', '>40.0', 'between 40.0 and 9888.5', '>=9888.5']
['<=5.0', '>5.0', 'between 5.0 and 158.5', '>=158.5']
['<=5.5', '>5.5', 'between 5.5 and 19.5', '>=19.5']
['<=0.5', '>0.5', 'between 0.5 and 39.5', '>=39.5']
['<=78.0', '<=36.0', 'between 36.0 and 78.0', '>78.0', 'between 78.0 and 14795.0', '>=14795.0']
['<=7.5', '<=0.5', 'between 0.5 and 7.5', '>7.5', 'between 7.5 and 26.5', '>=26.5']
['<=5.5', '>5.5', 'between 5.5 and 249.0', '>=249.0']
['<=737.0', '<=344.0', 'between 344.0 and 737.0', '>737.0', 'between 737.0 and 1531.0', '>=1531.0']
['<=266.5', '<=106.5', 'between 106.5 and 266.5', '>266.5', 'between 266.5 and 16298.0', '>=16298.0']
['<=620.0', '<=268.5', 'between 268.5 and 620.0', '>620.0', 'between 620.0 and 3601.5', '>=3601.5']
['<=59.5', '<=46.5

In [3]:
response='ABANDONED'
ct='New'
from feature_clustering import feature_clustering
i='pca'
df_no_pca=dummy_pca
df_no_pca,y, plsca= feature_clustering(df_no_pca,response, i)




sample dataframe shape is:  (50000, 1374)
{130: 45.572089983767597, 70: 45.965866466521661, 170: 45.435534672628286, 110: 45.670546680217356, 50: 46.306481331778549, 150: 45.616905789674682, 90: 45.937892387075408, 190: 45.471008307866811}
the best # of components is: 50


In [4]:
from clustering import obs_clustering
clust='True'
df=df_no_pca
df, df_w_labels, df_w_clust=obs_clustering(df,response,clust)


(50000, 45)
47.0991477466
47.1385492476
47.1654740047
15
8     45477
3     43229
7     43180
1     42523
13    40628
14    38515
0     35206
12    33274
2     32403
6     31802
5     27867
9     23242
4     23019
10    20375
11    19260
Name: LABELS, dtype: int64


In [5]:
from pca_dim_red import pca_code
i='False'
response='ABANDONED'
df_no_pca=df_w_labels
print list(df_no_pca)
df_pca, fitted_pca = pca_code(df_no_pca,response,i)



['LABELS', u'FILING_STATUS_HeadOfHousehold', "AMOUNT_HOPE_CREDIT_pcl_'<=5.5'", "MINDBENDER_REVENUE_pcl_'>=32.490001678466797'", 'PRS_SCORE_9.0', u'FED_FORM_TYPE_F1040A', "AGE_TAXPAYER_pcl_'>12.5'", 'PRS_SCORE_8.0', 'PRS_SCORE_7.0', u'CHANNEL_Paid Search - Brand', 'NUM_W2_-1', u'ENTRY_PAGE_GROUP_MyTurboTax', u'IMPORT_TYPE_Import', "STATE_REVENUE_pcl_'<=27.864999771118164'", "STATE_REVENUE_pcl_'<=29.994998931884766'", u'CHANNEL_Non-Campaign', "AGE_TAXPAYER_pcl_'<=12.5'", "AMOUNT_EXEMPTIONS_pcl_'<=3960.0'", 'NUM_W2_4.0', "AGE_TAXPAYER_pcl_'between 8.5 and 12.5'", u'START_SKU_600|Paid Deluxe', 'NUM_W2_1.0', u'CHANNEL_Organic Search', u'ENTRY_PAGE_GROUP_Product and Pricing Pages', 'PRS_SCORE_10.0', u'FED_FORM_TYPE_F1040EZ', 'REJECT_COUNT_1.0', 'REJECT_COUNT_0.0', 'PRS_SCORE_-1', u'DMA_AREA_Los Angeles', 'IMPORT_TYPE_-1', u'CHANNEL_Paid Search - Generic', u'ENTRY_PAGE_GROUP_Homepage', "AMOUNT_EITC_pcl_'<=1573.5'", "NON_CA_REFUND_TRANSFER_REVENUE_pcl_'>33.165000915527344'", "MINDBENDER_REVENU

In [6]:
from get_arrays import get_arrays
i='False'
j='False'
dummy_pca=df_pca
response = 'ABANDONED'
x, y = get_arrays(dummy_pca, response, i, j)

In [7]:
from model_algorithm import algorithm
response = 'ABANDONED'
models, name = algorithm(x, y, response)

c:\python27\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


logistic_regression 0.585981499165 0.248593887064 0.356850980828 0.5785158177 83450 8411 43686 14453
                                                index       sig
0                                           NUM_W2_-1  0.911163
1                            AGE_TAXPAYER_pcl_'>12.5'  0.385968
2   NON_CA_REFUND_TRANSFER_REVENUE_pcl_'>33.165000...  0.352445
3             AGE_TAXPAYER_pcl_'between 8.5 and 12.5'  0.293484
4                                LAST_STATUS_NO EFILE  0.254505
5                       CHANNEL_Paid Search - Generic  0.227977
6                                    REJECT_COUNT_1.0  0.208510
7                    AMOUNT_EXEMPTIONS_pcl_'<=3960.0'  0.207912
8                                CHANNEL_Non-Campaign  0.163356
9                                      IMPORT_TYPE_-1  0.145129
10                               DMA_AREA_Los Angeles  0.134497
11                              FED_FORM_TYPE_F1040EZ  0.132610
12                             ENTRY_PAGE_GROUP_Other  0.130157
13 

ValueError: X has 286 features per sample; expecting 44

In [ ]:
import pandas as pd
import numpy as np
from predicting_code import transform_to_pca, bin_pca_score_set, get_scoring_arrays, score_set_feature_selection, predict,get_scoring_arrays
defection=pd.DataFrame()
i, j, k=['False','True','False' ]
df_trans_pca = transform_to_pca(scoring_df, fitted_pca, i)
# print df_trans_pca
scoring_df_trans = bin_pca_score_set(df_trans_pca, length_dict, j)

print scoring_df_trans.shape, x.shape
a=list(x)
b=list(scoring_df_trans)
final=list(set(b).difference(a))
scoring_df_trans.drop(final, axis=1, inplace=True)
print scoring_df_trans.shape
x.drop(list(scoring_df_trans), axis=1, inplace=True)
print x.shape
c=list(x)

m=scoring_df_trans.shape[0]     
n= x.shape[1]
print  scoring_df_trans.shape, x.shape

zeros_df=pd.DataFrame(np.zeros((m, n)), columns=c)



scoring_df_trans.reset_index(['CUSTOMER_KEY'], inplace=True)

final_scoring_df=pd.concat([zeros_df,scoring_df_trans ], axis=1)
final_scoring_df.set_index('CUSTOMER_KEY', inplace=True)
print final_scoring_df.shape

scoring_df_trans=final_scoring_df
x_score, index=score_set_feature_selection(scoring_df_trans,plsca, k)
x_score.set_index('CUSTOMER_KEY', inplace=True)
df_p=predict(models,name,x_score, index)
defection=pd.concat([df_p, defection], axis=1)
print defection.head()


In [ ]:
from data_intersection import scoring_data_intersection
data = scoring_data_intersection(df_no_pca, scoring_df)
# print data.head()
print df_cont